# DWD Population Plot
Reads LDC galaxy key file and produces plot of DWD number density in GW frequency/amplitude plane, overlaid with LISA and AMIGO sensitivity curves


### Setup

In [ ]:
# Import standard libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.ticker import FormatStrFormatter
import sys
from lisacattools import HPhist, convert_ecliptic_to_galactic

# import GW Imager libraries
sys.path.append('../src/')
import constants
import metrics
import sources
import concepts
import subsystems
import background
import falseColorGW

# make directory for plots
!mkdir -p '../plots/'

fontSize = 20


### Read in keyfile and make number density

In [ ]:
# Load in the GBs from LDC
LDCgalaxyFile = **FILENAME**
galaxy = pd.read_hdf(LDCgalaxyFile,key='keyfile')
galaxy.describe()

In [ ]:
# Perform 2D histogram in frequency/ampltiude space

logf = np.log10(np.array(galaxy['Frequency']))
logf_bin_edges = np.linspace(-4.5,-1.5,81)
logf_bin_centers = logf_bin_edges[0:-1]+0.5*(logf_bin_edges[1:]-logf_bin_edges[0:-1])
bin_fwidth = 10**(logf_bin_edges[1:])-10**(logf_bin_edges[0:-1])

logA = np.log10(np.array(galaxy['Amplitude']))
logA_bin_edges = np.linspace(-26,-20,101)
logA_bin_centers = logA_bin_edges[0:-1]+0.5*(logA_bin_edges[1:]-logA_bin_edges[0:-1])
bin_Awidth = 10**(logA_bin_edges[1:])-10**(logA_bin_edges[0:-1])

bin_areas = np.outer(bin_fwidth,bin_Awidth)

counts,_,_ = np.histogram2d(logf,logA, [logf_bin_edges,logA_bin_edges])

DWD_density = np.transpose(counts/bin_areas)

### Detector models

In [ ]:
# Define mission concept
lisa = concepts.LISASciRDv1
lisa['label']= 'LISA'

lisa_bg = background.add2model(lisa)
lisa_bg['label'] = 'LISA + foreground'

amigo = concepts.AMIGO
amigo['label']= 'AMIGO'

amigo_bg = background.add2model(amigo)
amigo_bg['label'] = 'AMIGO + foreground'

logf = np.linspace(-4,0,512)
S_lisa = np.log10(np.sqrt(metrics.makeSensitivity(10**logf,lisa)/(4*constants.year)))
S_lisa_bg = np.log10(np.sqrt(metrics.makeSensitivity(10**logf,lisa_bg)/(4*constants.year)))
S_amigo = np.log10(np.sqrt(metrics.makeSensitivity(10**logf,amigo)/(4*constants.year)))
S_amigo_bg = np.log10(np.sqrt(metrics.makeSensitivity(10**logf,amigo_bg)/(4*constants.year)))


### Make the Plot

In [ ]:
# Make Plot
fig = plt.figure(figsize=(14, 10), dpi = 100)
ax = plt.axes()


#cNorm = colors.LogNorm(vmin=galaxy_detected['CDL'].min(), vmax=galaxy_detected['CDL'].max()) #re-wrapping normalization
vmin=10**(np.floor(np.log10(np.min(DWD_density[DWD_density.nonzero()]))))
vmax=10**(np.ceil(np.log10(np.max(DWD_density))))
cNorm = colors.LogNorm(vmin=vmin,vmax=vmax)

scalarMap = cm.ScalarMappable(norm=cNorm, cmap=plt.cm.get_cmap('plasma'))

ax.pcolormesh(logf_bin_centers,logA_bin_centers,np.log10(DWD_density),cmap='plasma',label='DWD density')
#ax.set_yscale('log')
#ax.set_xscale('log')
ax.set_xlim([-4.5,-1.5])
ax.set_ylim([-26,-20])
ax.grid()


def format_func(value, tick_number):
    return (r"$10^{%g}$" % value)

ax.set_xticks([-4,-3,-2])
ax.set_yticks([-25,-23,-21])

ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.tick_params(axis = 'both', which = 'major', labelsize = fontSize)

plt.xlabel('Frequency [Hz]',fontsize=fontSize)
plt.xticks(fontsize=fontSize)
plt.ylabel('Strain Amplitude',fontsize=fontSize)
plt.yticks(fontsize=fontSize)

ax.plot(logf,S_lisa, color='k',linestyle = '--',label=lisa['label'])
ax.plot(logf,S_lisa_bg, color='k',linestyle = '-',label=lisa_bg['label'])
ax.plot(logf,S_amigo, color='gray',linestyle = '--',label=amigo['label'])
ax.plot(logf,S_amigo_bg, color='gray',linestyle = '-',label=amigo_bg['label'])

ax.legend(fontsize=fontSize,framealpha=0.9,loc='lower right')


cbar = fig.colorbar(scalarMap)
cbar.set_label('DWD density [Hz$^{-1}$]',fontsize=fontSize)
cbar.ax.tick_params(labelsize=fontSize)

fig.savefig('../plots/DWDdensity.png')